In [2]:
"""
GUI Application: Product Recommendations Based on Last Purchased Item
-------------------------------------------------------------------
This script wraps your existing recommendation logic in a desktop window so a user can:
  • Enter a Customer ID
  • (Optionally) choose how many recommendations (Top N)
  • See the last purchased item and its recommended similar products in the same window
  • Close / minimize using standard window controls

Technologies Used:
  - pandas (data handling)
  - scikit-learn (cosine similarity)
  - tkinter (built‑in Python GUI toolkit, no extra install needed)
  - threading (to keep the UI responsive during initial data load)

How It Works:
  1. On startup, a background thread loads the Excel file, cleans data, builds the pivot matrix, and computes the item similarity matrix.
  2. While loading, the "Get Recommendations" button is disabled and a status label shows progress.
  3. After loading finishes, user enters a Customer ID (numeric, e.g., 12347) and clicks the button (or presses Enter).
  4. The app finds that customer's last purchased item and pulls Top N similar items from the similarity matrix.
  5. Results are displayed in a scrollable text area.

Optional Enhancements (commented sections):
  - Add a second tab for searching by Product Description instead of Customer ID.
  - Cache recommendations (simple dict) to avoid repeated lookups.
  - Export displayed recommendations to CSV.

Pack into an EXE (optional):
  pyinstaller --onefile --noconsole recommendation_gui.py

"""

import threading
import tkinter as tk
from tkinter import ttk, messagebox
from tkinter.scrolledtext import ScrolledText
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import os

# ---------------- Configuration ---------------- #
DATA_PATH = r"E:\c drive\project\data\online_retail\online_retail_II.xlsx"  # <-- Adjust if needed
TOP_N_DEFAULT = 5

# ---------------- Global (populated after load) ---------------- #
df = None
item_sim_df = None
loaded = False

# ---------------- Data Loading Logic ---------------- #

def load_data():
    global df, item_sim_df, loaded
    try:
        update_status("Loading Excel data ...")
        raw = pd.read_excel(DATA_PATH)
        update_status("Cleaning data ...")
        # Basic cleaning
        raw = raw[(raw['Quantity'] > 0) & (raw['Price'] > 0)]
        raw.dropna(subset=['Customer ID', 'Description'], inplace=True)
        # Ensure consistent types
        raw['Customer ID'] = raw['Customer ID'].astype(float)
        raw['TotalPrice'] = raw['Quantity'] * raw['Price']
        if 'InvoiceDate' in raw.columns:
            raw['InvoiceDate'] = pd.to_datetime(raw['InvoiceDate'])
        else:
            raise ValueError("Column 'InvoiceDate' not found in dataset.")

        update_status("Building pivot (this may take a moment) ...")
        pivot = raw.pivot_table(index='Customer ID', columns='Description', values='TotalPrice', aggfunc='sum', fill_value=0)

        update_status("Computing cosine similarity matrix ...")
        similarity_matrix = cosine_similarity(pivot.T)
        item_sim_df_local = pd.DataFrame(similarity_matrix, index=pivot.columns, columns=pivot.columns)

        # Assign to globals atomically near end
        df = raw
        item_sim_df = item_sim_df_local
        loaded = True
        update_status("✅ Data loaded. Ready.")
        enable_inputs()
    except FileNotFoundError:
        update_status("❌ File not found.")
        messagebox.showerror("Error", f"File not found:\n{DATA_PATH}")
    except Exception as e:
        update_status("❌ Load failed.")
        messagebox.showerror("Error", f"Failed to load data: {e}")

# ---------------- Recommendation Logic ---------------- #

def recommend_for_customer(customer_id: float, top_n: int = TOP_N_DEFAULT):
    if not loaded:
        return "Data still loading..."

    global df, item_sim_df
    if customer_id not in df['Customer ID'].values:
        return f"❌ Customer ID {customer_id} not found."

    cust_orders = df[df['Customer ID'] == customer_id].sort_values('InvoiceDate')
    if cust_orders.empty:
        return f"❌ No orders found for Customer ID {customer_id}."

    last_item = cust_orders.iloc[-1]['Description']
    if last_item not in item_sim_df.index:
        return f"⚠️ Last item '{last_item}' not in similarity index."

    similar_series = item_sim_df[last_item].sort_values(ascending=False).drop(last_item)
    top_items = similar_series.head(top_n).index.tolist()

    lines = [f"Customer ID: {int(customer_id) if customer_id.is_integer() else customer_id}",
             f"🛍 Last Purchased: {last_item}",
             f"Top {top_n} Recommended Products:"]
    for i, itm in enumerate(top_items, 1):
        lines.append(f"  {i}. {itm}")
    return "\n".join(lines)

# ---------------- GUI Setup ---------------- #

root = tk.Tk()
root.title("Product Recommendation Viewer")
root.geometry("700x500")
root.minsize(650, 500)

# (Optional) Set an icon if you have one
# try: root.iconbitmap('app.ico') except: pass

main_frame = ttk.Frame(root, padding=12)
main_frame.pack(fill=tk.BOTH, expand=True)

# Title Label
title_lbl = ttk.Label(main_frame, text="Recommendations Based on Last Purchased Item", font=("Segoe UI", 14, "bold"))
title_lbl.pack(anchor="center", pady=(0, 10))

# Input Frame
input_frame = ttk.Frame(main_frame)
input_frame.pack(fill=tk.X, pady=4)

cust_id_label = ttk.Label(input_frame, text="Customer ID:")
cust_id_label.pack(side=tk.LEFT)

cust_id_var = tk.StringVar()
cust_id_entry = ttk.Entry(input_frame, textvariable=cust_id_var, width=15)
cust_id_entry.pack(side=tk.LEFT, padx=5)

# Top N selector
n_label = ttk.Label(input_frame, text="Top N:")
n_label.pack(side=tk.LEFT, padx=(15, 0))

n_var = tk.StringVar(value=str(TOP_N_DEFAULT))
n_spin = ttk.Spinbox(input_frame, from_=1, to=20, textvariable=n_var, width=5)
n_spin.pack(side=tk.LEFT, padx=5)

# Recommend Button
recommend_btn = ttk.Button(input_frame, text="Get Recommendations")
recommend_btn.pack(side=tk.LEFT, padx=(15, 0))

# Status Label
status_var = tk.StringVar(value="Initializing ...")
status_lbl = ttk.Label(main_frame, textvariable=status_var, foreground="#555")
status_lbl.pack(fill=tk.X, pady=(6, 2))

# Results Text Area
results_box = ScrolledText(main_frame, wrap=tk.WORD, height=18, font=("Consolas", 10))
results_box.pack(fill=tk.BOTH, expand=True, pady=(6, 0))
results_box.insert(tk.END, "Waiting for data to load...\n")
results_box.configure(state=tk.DISABLED)

# Footer Hint
hint_lbl = ttk.Label(main_frame, text="Enter Customer ID and press 'Get Recommendations' or Enter.", foreground="#666")
hint_lbl.pack(anchor="w", pady=(4, 0))

# ---------------- Helper Functions ---------------- #

def update_status(msg: str):
    status_var.set(msg)
    status_lbl.update_idletasks()

def enable_inputs():
    recommend_btn.config(state=tk.NORMAL)
    cust_id_entry.config(state=tk.NORMAL)
    n_spin.config(state=tk.NORMAL)

def disable_inputs():
    recommend_btn.config(state=tk.DISABLED)
    cust_id_entry.config(state=tk.DISABLED)
    n_spin.config(state=tk.DISABLED)

# Initially disable until data loads
disable_inputs()

# ---------------- Event Handlers ---------------- #

def on_get_recommendations(event=None):
    if not loaded:
        messagebox.showinfo("Please wait", "Data is still loading.")
        return

    cid_raw = cust_id_var.get().strip()
    if cid_raw == "":
        messagebox.showwarning("Input Needed", "Please enter a Customer ID.")
        return
    try:
        cid = float(cid_raw)
    except ValueError:
        messagebox.showerror("Invalid Input", "Customer ID must be numeric.")
        return

    try:
        top_n = int(n_var.get())
        if top_n < 1:
            raise ValueError
    except ValueError:
        messagebox.showerror("Invalid Input", "Top N must be a positive integer.")
        return

    output = recommend_for_customer(cid, top_n)
    results_box.configure(state=tk.NORMAL)
    results_box.delete(1.0, tk.END)
    results_box.insert(tk.END, output + "\n")
    results_box.configure(state=tk.DISABLED)

recommend_btn.config(command=on_get_recommendations)
# Bind Enter key to trigger recommendations
cust_id_entry.bind('<Return>', on_get_recommendations)
root.bind('<Return>', on_get_recommendations)  # global Enter

# ---------------- Start Background Loading ---------------- #

def start_loading_thread():
    thread = threading.Thread(target=load_data, daemon=True)
    thread.start()

start_loading_thread()

# ---------------- Run App ---------------- #
if __name__ == '__main__':
    root.mainloop()
